# 02 - Event Study: US–Venezuela Strike 3 Jan 2026

Tujuan:
- Menghitung abnormal return (AR) dan cumulative abnormal return (CAR)
  di sekitar event 3 Januari 2026 untuk:
  - Brent, WTI
  - EEM, ILF
  - BTC, ETH

Event:
- 3 Januari 2026: Serangan udara AS ke Venezuela dan penangkapan Nicolás Maduro. [web:18][web:31][web:33]

Metode:
- Market model dengan S&P 500 (^GSPC) sebagai market index.
- Estimation window: t0-60 s/d t0-10
- Event window: t0-3 s/d t0+3


In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime

returns = pd.read_csv("returns_master.csv", parse_dates=["Date"], index_col="Date")
returns.head()

,Brent,WTI,EEM,ILF,BTC,ETH,SP500
Date,,,,,,,
2025-10-02,-0.019157,-0.021267,0.004634,-0.004933,0.016984,0.030959,0.000618
2025-10-03,0.006530,0.006592,0.002955,0.002117,0.013051,0.005986,0.000066
2025-10-04,0.000000,0.000000,0.000000,0.000000,0.001299,-0.005703,0.000000
2025-10-05,0.000000,0.000000,0.000000,0.000000,0.008848,0.005825,0.000000
2025-10-06,0.014462,0.013217,0.004599,-0.005656,0.009982,0.037458,0.003640


2.3. Definisikan event date dan window

In [4]:
event_date = pd.Timestamp("2026-01-03")  # t0

estimation_start = event_date - pd.Timedelta(days=80)
estimation_end = event_date - pd.Timedelta(days=10)

event_window_start = event_date - pd.Timedelta(days=3)
event_window_end = event_date + pd.Timedelta(days=3)

assets = ["Brent", "WTI", "EEM", "ILF", "BTC", "ETH"]
market_col = "SP500"

# helper untuk market

In [5]:
def estimate_market_model(asset_returns, market_returns):
    X = sm.add_constant(market_returns)
    model = sm.OLS(asset_returns, X).fit()
    alpha = model.params["const"]
    beta = model.params[market_col]
    return alpha, beta, model


# Loop semua market untuk hitung AR abnormal return dan CAR 

In [6]:
results = []

for asset in assets:
    # Ambil estimation window
    est_data = returns.loc[estimation_start:estimation_end, [asset, market_col]].dropna()
    ev_data = returns.loc[event_window_start:event_window_end, [asset, market_col]].dropna()
    
    if est_data.empty or ev_data.empty:
        continue
    
    alpha, beta, model = estimate_market_model(est_data[asset], est_data[market_col])
    
    # Expected return di event window
    X_ev = sm.add_constant(ev_data[market_col])
    expected = alpha + beta * ev_data[market_col]
    
    # Abnormal return
    ar = ev_data[asset] - expected
    
    # CAR untuk beberapa sub-window: [-1,1], [-3,3]
    window = ar.index
    t0 = event_date
    
    # Helper ambil subset
    ar_m3_p3 = ar.loc[event_window_start:event_window_end].sum()
    ar_m1_p1 = ar.loc[event_date - pd.Timedelta(days=1):event_date + pd.Timedelta(days=1)].sum()
    
    results.append({
        "asset": asset,
        "CAR_-3_+3": ar_m3_p3,
        "CAR_-1_+1": ar_m1_p1
    })

car_df = pd.DataFrame(results)
car_df

,asset,CAR_-3_+3,CAR_-1_+1
0,Brent,-0.017602,-0.001559
1,WTI,-0.009121,-0.001507
2,EEM,0.028913,0.025752
3,ILF,0.003450,0.007050
4,BTC,0.061157,0.038108
5,ETH,0.096171,0.057800


# SAVE CAR

In [7]:
car_df.to_csv("event_study_CAR.csv", index=False)